In [ ]:
import re     # работа с регулярными выражениями
import os
import nltk   # для обработки естественного языка
import pandas as pd
from pathlib import Path
from nltk import SnowballStemmer, WordNetLemmatizer # для стемминга и лемматизации
from nltk.corpus import wordnet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# стемминг - обрезания слова до его основы
# SnowballStemmer полезен при анализе текстов
# для уменьшения количества уникальных слов и
# упрощения анализа. Он работает на основе
# алгоритма Портера и может обрабатывать слова
# на разных языках.
stemmer = SnowballStemmer("english")

# лемматизация - приведения слова к его базовой форме
# WordNetLemmatizer может уменьшить количество уникальных
# слов в тексте и упростить анализ. Он использует базу
# данных WordNet для определения базовой формы
# слова в зависимости от его части речи.
lemmatizer = WordNetLemmatizer()

In [ ]:
# знаки препинания, обозначающие конец предложения
end_of_clause = ['.', '?', '!', '...']

# принимает слово и возвращает его часть речи
def get_wordnet_pos(word):
    # pos_tag принимает список слов и возвращает список кортежей,
    # каждый из которых содержит слово и его POS-тег
    tag = nltk.pos_tag([word])[0][1][0].upper()
    # сопоставление POS-тега с тегом WordNet
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    # если тег не найден, то возвращает существительное
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
# словарь knownAbbrevs, содержащий сокращения и их полные формы
knownAbbrevs = {
    "col.": "college",
    "messrs.": "messieurs",
    "gov.": "government",
    "adm.": "admiral",
    "rev.": "revolution",
    "fr.": "french",
    "maj.": "major",
    "sgt.": "sergeant",
    "cpl.": "corporal",
    "pvt.": "private",
    "capt.": "captain",
    "ave.": "avenue",
    "pres.": "president",
    "brig.": "brigadier",
    "cmdr.": "commander",
    "asst.": "assistant",
    "assoc.": "associate",
    "insp.": "inspiration",
    "st.": "saint",
    "dr.": "doctor",
    "tel.": "telephone",
    "no.": "number",
    "u.s.": "United States",
    "u.k.": "United Kingdom",
    "prof.": "professor",
    "inc.": "incorporated",
    "ltd.": "limited",
    "corp.": "corporation",
    "co.": "corporation",
    "mr.": "mister",
    "plc.": "Public Limited Company",
    "assn.": "association",
    "univ.": "university",
    "intl.": "international",
    "sys.": "system",
    "est.": "Eastern Standard Time",
    "ext.": "extention",
    "sq.": "square",
    "jr.": "junior",
    "sr.": "senior",
    "bros.": "brothers",
    "ed.d.": "Doctor of Education",
    "ph.d.": "Doctor of Phylosophy",
    "sci.": "Science",
    "etc.": "Et Cetera",
    "al.": "al",
    "seq.": "sequence",
    "orig.": "original",
    "incl.": "include",
    "eg.": "eg",
    "avg.": "average",
    "pl.": "place",
    "min.": "min",
    "max.": "max",
    "cit.": "citizen",
    "mrs.": "mrs",
    "mx.": "mx",
    "miss.": "miss",
    "atty.": "attorney"
}

In [ ]:
# регулярные выражения
tokens = [
    # аббревиатуры
    ["abbrev", "|".join(map(lambda kv: "(?i:" + re.escape(kv[0]) + ")", knownAbbrevs.items()))],
    # IP-адреса
    ["ipaddress", "[0-9]+\\.[0-9]+\\.[0-9]+\\.[0-9]+"],
    # числительные (5th)
    ["numeral", "[0-9]+((th)|(\\'s))"],
    # цифры с дефисом и буквами (123A, 456-B, 789c)
    ["metrics", "[0-9]+(-)?[A-Z]?[a-z]+"],
    # последовательность с $ ($2,525)
    ["currency", "\\$[0-9]+(\\,[0-9]+)*"],
    # числа (в том числе дробные)
    ["number", "[0-9]+(.|,|-)[0-9]*"],
    # пробелы, новая строка, слэш, табуляция
    ["whitespace", "\\s|\\n|\\\\|\\t"],
    # скобки
    ["braces", "\\(|\\)"],
    # слова в кавычках
    ["quoted", "(\\\")[^\\\"]*(\\\")"],
    # пунктуация (, . ? ! ...)
    ["punct", ",|\\.|\\?|\\!|(\\.\\.\\.)"],
    # обычные слова, с апострофом (won't), с дефисом (twenty-five)
    ["word", "[A-Za-z][A-Za-z\\']*(-[A-Z\\']?[A-Za-z\\']+)*"],
    # одиночные символы, символы, которые не являются буквенно-цифровыми
    ["other", ".[^a-zA-Z0-9]*"],
    # телефонные номера, например: +7-901-000-00-00, 8(918)3213412 и т.д.
    ["mobile_phone", "^(?:\+7|8)(?:(?:-\d{3}-|\(\d{3}\))\d{3}-\d{2}-\d{2}|\d{10})"],
    # электронная почта, например: abc@abc.abc
    ["mail", "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"],
]

In [ ]:
# компиляция регулярного выражения, которое будет использоваться
# для идентификации токенов на основе заданных выражений
regex = re.compile("^(" + "|".join(map(lambda t: "(?P<" + t[0] + ">" + t[1] + ")", tokens)) + ")")

# словарь с классами новостей
classes = dict([
    (1, 'World'),
    (2, 'Sports'),
    (3, 'Business'),
    (4, 'Sci-Tech')
])

In [ ]:
# токенизация
def tokenize_text(text):
    # позиция в тексте
    pos = 0
    s = text
    # найденные токены
    line = []
    while len(s) > 0:
        # поиск первого совпадения с регулярным выражением
        match = regex.search(s)
        # проверка, что совпадение найдено и конечная позиция совпадения больше начальной позиции
        if match and match.endpos > match.pos:
            for gr in tokens:
                # только те значения из группы захвата, которые не являются None
                tt = list(filter(lambda kv: kv[1] is not None, match.groupdict().items()))
                # проверка, что список tt содержит ровно один элемент
                if len(tt) == 1:
                    # значение ключа токена
                    kind = tt[0][0]
                    # значение значения токена
                    part = tt[0][1]
                    # случай работы с аббревиатурами
                    if kind == 'abbrev':
                        kind = 'word'
                        part = knownAbbrevs[part.lower()]
                    # новый элемент в список, состоящий из: позиции в тексте, типа токена и самого токена
                    line.append([pos, kind, part])
                    # увеличение значения переменной pos на длину токена
                    pos += len(tt[0][1])
                    # урезание с конца на длину токена
                    s = s[len(tt[0][1]):]
                    break
                else:
                    print('tokenization is not possible: ' + s)
        else:
            print('tokenization is not possible: ' + s)
    return line

In [ ]:
def operation_with_file(fname):
    print('working on ', fname)
    df = pd.read_csv(fname, sep=',', header=None)
    data = df.values
    data_count = len(data)
    # номер обрабатываемого файла
    n = 0
    for row in data:
        class_id = row[0]
        try:
            dir_path = "/content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/" + Path(fname).name.split('.')[0] + "/" + classes[class_id] + '/'
            if not os.path.exists(dir_path):
                os.makedirs(dir_path)
            f = open(dir_path + str(n) + '.tsv', 'w+')
            f.truncate(0)
            # перебор столбцов, кроме первого, в текущей строке данных
            for i in range(1, len(row)):
                #конкретная ячейка
                text = row[i]
                tokens = tokenize_text(text)
                prev = [0, '', '']
                # перебор токенов
                for w in tokens:
                    # проверка, что тип токена не пробел
                    if w[1] != 'whitespace':
                        # запись строки со значениями типа токена, самого токена, стеммы токена, леммы токена
                        f.write(w[1] + '\t' + w[2] + '\t' + stemmer.stem(w[2]) + "\t" + lemmatizer.lemmatize(w[2], get_wordnet_pos(w[2])) + '\n')
                    # проверка, что последний токен (prev[2]) находится в списке end_of_clause (конец предложения)
                    elif prev[2] in end_of_clause:
                        f.write('\n')
                    # текущий токен
                    prev = w
                f.write('\n')
            f.close()
        except Exception as e:
            # печать ошибки
            print(e)
            # вывод информации о номере, тексте и токенах, связанных с возникшей ошибкой
            print([n, text, tokens])
            pass
        n = n + 1
        # значение n кратно 1000
        if n % 1000 == 0:
            # печать прогресса обработки файла в %
            print(int(n * 100 / data_count), '%')

In [ ]:
path_train = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ITMO', 'sem_3', 'NLP', 'assets', 'dataset', 'train.csv')
path_test = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ITMO', 'sem_3', 'NLP', 'assets', 'dataset', 'test.csv')

In [ ]:
operation_with_file(path_train)
# operation_with_file(path_test)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# path_train = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ITMO', 'sem_3', 'NLP', 'assets', 'annotated-corpus', 'train')
# path_test = os.path.join(os.getcwd(), 'drive', 'MyDrive', 'ITMO', 'sem_3', 'NLP', 'assets', 'annotated-corpus', 'test')

# !zip -r /content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/test /content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/test
# !zip -r /content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/train /content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/train


# from google.colab import files
# files.download("/content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/train.zip")
# files.download("/content/drive/MyDrive/ITMO/sem_3/NLP/assets/annotated-corpus/test.zip")
